In [1]:
import lib.PrimeGen as pg

In [2]:
import math
import gmpy2
import random

In [3]:
class RSA:
    def __init__(self, bits):
        pgen = pg.PrimeGen()
        p = pgen.Generate(bits) # Generate a *safe prime* with __bits__ BITS.
        print ('Computed Random Prime p.')
        q = pgen.Generate(bits) # Generate a *safe prime* with __bits__ BITS.
        print ('Computed Random Prime q.')
        phi = (p-1)*(q-1)       # euler's phi function for n
        
        self.n = p * q
        self.e = 2**16 + 1
        self.__d = gmpy2.invert(self.e, phi)
        print ('RSA-%d Initiated Successfully.' %(bits))
    
    def __padhexa(self, s, l):
        return '0x' + s[2:].zfill(l)
        
    def __GenerateByteSequence(self, byte_length):
        hex_arr = list(map(str,\
                       list(range(1, 10)) + ['A', 'B', 'C', 'D', 'E', 'F']))
        secure_random = random.SystemRandom()
        tmp = byte_length * 8
        res = ''
        while (tmp is not 0):
            word = secure_random.choice(hex_arr)
            size = 4
            if (size <= tmp):
                tmp -= size
                res += word
        return res
        
    def __PadMessage(self, plainText, n):
        EB = math.ceil(n.bit_length()/8) # byte-size of Encryption Block (EB)
        DB = math.ceil(plainText.bit_length()/8)       # byte-size of Data Block (DB)
        PS = EB - 3 - DB                        # byte-size of Padding String (PS)
        if not(DB <= EB - 11):
            raise ValueError('Message > Encryption Block - 11.')
        else:
            PaddingString = self.__GenerateByteSequence(PS)
        paddedText = '0x0002'+PaddingString+'00'+hex(plainText)[2:]
        return paddedText
        
    def __PublicKey(self):
        return self.e, self.n
    
    def __PrivateKey(self):
        return self.__d, self.n
    
    def Encrypt(self, plainText, padding=False):
        [e, n] = self.__PublicKey()
        if padding:
            plainTextPadded = self.__PadMessage(plainText, n)
            tmp = pow(int(plainTextPadded,16), e, n)
        else:
            tmp = pow(plainText, e, n)
        return tmp
    
    def Decrypt(self, cipherText, padding=False):
        [d, n] = self.__PrivateKey()
        
        decipherText = pow(cipherText, d, n)
        
        if padding:
            byte_difference = abs(len(hex(rsa256.n)) - len(hex(decipherText)))
            if (byte_difference >= 3):
                bytelen = len(hex(decipherText)[2:])
                decipherText = self.__padhexa(hex(decipherText), bytelen + 3)
            
            if (decipherText[2:6] == '0002'):
                decipherText = int(decipherText.split('00')[-1], 16)
            else:
                decipherText = ''
            
        return decipherText

In [4]:
rsa256 = RSA(128)

Computed Random Prime p.
Computed Random Prime q.
RSA-128 Initiated Successfully.


In [5]:
m1 = random.randrange(1024)
c1 = rsa256.Encrypt(m1, padding=True)
mm1 = rsa256.Decrypt(c1, padding=True)

print ('Initial Message: ', m1, '\n')
print ('Encrypted Message: ', c1, '\n')
print ('Decrypted Message: ', mm1, '\n')

Initial Message:  279 

Encrypted Message:  43356737391121762645629743442208744398871977148367364650521060544149888140669 

Decrypted Message:  279 



In [6]:
m2 = random.randrange(1024)
c2 = rsa256.Encrypt(m2, padding=True)
mm2 = rsa256.Decrypt(c2, padding=True)

print ('Initial Message: ', m2, '\n')
print ('Encrypted Message: ', c2, '\n')
print ('Decrypted Message: ', mm2, '\n')

# Show Homomorphic Properties
c3 = (c1 * c2) % rsa256.n
print (rsa256.Decrypt(c3, padding=True))

Initial Message:  765 

Encrypted Message:  1224470163388410177486002298728049660032562552903625763337074118391219937735 

Decrypted Message:  765 




In [7]:
rsa256.n

60788469777168206489955738712198866966511220273039441480741516390199707763139

# Make The PKCS #1 v1.5 Padded Cipher Text

In [12]:
EB = math.ceil(rsa256.n.bit_length()/8) # byte-size of Encryption Block (EB)
DB = math.ceil(m1.bit_length()/8)       # byte-size of Data Block (DB)
PS = EB - 3 - DB                        # byte-size of Padding String (PS)

hex_m1 = m1.to_bytes((m1.bit_length() + 7) // 8, byteorder='big')


print ('[Target] CipherText Length: ', EB)
print ('\nMessage: ', hex_m1)
print ('\nMessage Length: ', DB)
print ('\nPadding Length: ', PS)

if not(DB <= EB - 11):
    print('Can\'t Encrypt This.')
    sys.exit(0)
else:
    PaddingString = GenerateByteSequence2(PS)

print ('\nComputed Padding: ', PaddingString)
print ('\nComputed Padding Length: ', len(PaddingString))
cipher_padded = bytes.fromhex('0002') + PaddingString + bytes.fromhex('00') + hex_m1

print ('\nPadded CipherText: ', cipher_padded)
print ('\nPadded CipherText Length:\nbytelen(0x0002) + bytelen(PaddingString) + bytelen(0x00) + bytelen(message) = ',\
        len(bytes.fromhex('0002')) + len(PaddingString) + len(bytes.fromhex('00')) + len(hex_m1))


cipherTextInt = int.from_bytes(cipher_padded, byteorder='big')
cipher_encrypted = pow(cipherTextInt, rsa256.e, rsa256.n)

print ('\nEncrypted CipherText: ', cipher_encrypted)

[Target] CipherText Length:  32

Message:  b'\x01\x17'

Message Length:  2

Padding Length:  27

Computed Padding:  bytearray(b'\x05\r\x0e\x03\x0e\x0e\x05\x07\x08\x04\x0e\x08\x0b\x03\x06\n\x04\x01\n\r\x0f\x03\x0b\x08\x0e\t\x0f')

Computed Padding Length:  27

Padded CipherText:  b'\x00\x02\x05\r\x0e\x03\x0e\x0e\x05\x07\x08\x04\x0e\x08\x0b\x03\x06\n\x04\x01\n\r\x0f\x03\x0b\x08\x0e\t\x0f\x00\x01\x17'

Padded CipherText Length:
bytelen(0x0002) + bytelen(PaddingString) + bytelen(0x00) + bytelen(message) =  32

Encrypted CipherText:  26127420999957549089656293076923861521958942870915209487015636757192480784695


In [13]:
plain_1 = rsa256.Decrypt(cipher_encrypted)

In [14]:
# print (EB)
# print (math.ceil(plain.bit_length()/8))
# print ('0x%0*x' % (len(hex(rsa256.n)[2:]), plain))
print(plain_1 == int(plain_1))
hex_cipher = int(plain_1).to_bytes((rsa256.n.bit_length() + 7) // 8, byteorder='big')

print(hex_cipher)
# print(type(hex_cipher))
# if (len(hex(plain_1)) < len(hex(rsa256.n))):
#     tmp4 = padhexa(hex(plain_1), len(hex(plain_1)[2:]) + 3)
if (hex_cipher[0:2] == b'\x00\x02'):
    print(int.from_bytes(hex_cipher.split(bytearray(1))[-1], byteorder='big'))
else: 
    print ('Wrong Padding')

True
b'\x00\x02\x05\r\x0e\x03\x0e\x0e\x05\x07\x08\x04\x0e\x08\x0b\x03\x06\n\x04\x01\n\r\x0f\x03\x0b\x08\x0e\t\x0f\x00\x01\x17'
279


In [18]:
EB = math.ceil(rsa256.n.bit_length()/8) # byte-size of Encryption Block (EB)
DB = math.ceil(m2.bit_length()/8)       # byte-size of Data Block (DB)
PS = EB - 3 - DB                        # byte-size of Padding String (PS)

print ('[Target] CipherText Length: ', EB)
print ('\nMessage: ', hex(m2))
print ('\nMessage Length: ', DB)
print ('\nPadding Length: ', PS)


if not(DB <= EB - 11):
    print('Can\'t Encrypt This.')
    sys.exit(0)
else:
    PaddingString = GenerateByteSequence(PS)

print ('\nComputed Padding: ', PaddingString)
print ('\nComputed Padding Length: ', math.ceil(len(bin(int(PaddingString, 16))[2:])/8))

cipher_padded = '0x0002'+PaddingString+'00'+hex(m2)[2:]

print ('\nPadded CipherText: ', cipher_padded)
print ('\nPadded CipherText Length:\nbytelen(0x0002) (=2) + bytelen(PaddingString) (=60) + bytelen(0x00) (=1) + bytelen(message) (=1) = ',\
       2 + math.ceil(len(bin(int(PaddingString, 16))[2:])/8) + 1 + math.ceil(len(bin(m2)[2:])/8))


cipher_encrypted_2 = pow(int(cipher_padded,16), rsa256.e, rsa256.n)
print ('\nEncrypted CipherText: ', cipher_encrypted_2)

[Target] CipherText Length:  64

Message:  0x362

Message Length:  2

Padding Length:  59

Computed Padding:  F43ABD4AF53B4DCFB8B3174BB49A5DEA25944B72A519F2D4A3E7AAEADFE7D2E8D7A117DD9B3CB7FA535CAFD8BE8644915FDF64511A9ED5AC1156D9

Computed Padding Length:  59

Padded CipherText:  0x0002F43ABD4AF53B4DCFB8B3174BB49A5DEA25944B72A519F2D4A3E7AAEADFE7D2E8D7A117DD9B3CB7FA535CAFD8BE8644915FDF64511A9ED5AC1156D900362

Padded CipherText Length:
bytelen(0x0002) (=2) + bytelen(PaddingString) (=60) + bytelen(0x00) (=1) + bytelen(message) (=1) =  64

Encrypted CipherText:  3102166782669551284215508552002243022823180267192622323751686012530241182121001869866637565342253514507997659482150754413987442387103710304740646821677168


In [19]:
plain_2 = rsa256.Decrypt(cipher_encrypted_2)

In [21]:
hex(plain_2)

'0x2f43abd4af53b4dcfb8b3174bb49a5dea25944b72a519f2d4a3e7aaeadfe7d2e8d7a117dd9b3cb7fa535cafd8be8644915fdf64511a9ed5ac1156d900362'

In [45]:
print(len(hex(plain_2)))
print(len(hex(rsa256.n)))


if (abs(len(hex(rsa256.n)) - len(hex(plain_2))) >= 3):
    tmp4 = padhexa(hex(plain_2), len(hex(plain_2)[2:]) + 3)

    
print(tmp4[2:6])
    
int(tmp4.split('00')[-1], 16)

126
130
0002


866

In [9]:
def padhexa(s, l):
    return '0x' + s[2:].zfill(l)

In [10]:
def GenerateByteSequence(byte_length):
    hex_arr = list(map(str,\
                   list(range(1, 10)) + ['A', 'B', 'C', 'D', 'E', 'F']))
    
    secure_random = random.SystemRandom()
    tmp = byte_length * 8
    res = ''
    while (tmp is not 0):
        word = secure_random.choice(hex_arr)
        size = 4
        if (size <= tmp):
            tmp -= size
            res += word
    return res

In [11]:
def GenerateByteSequence2(byte_length):
    byteSeq = bytearray(byte_length)
    for byte in range(byte_length):
        byteSeq[byte] = random.randint(1, 15)
    return byteSeq